<a href="https://colab.research.google.com/github/MaimunaSun/NLP/blob/main/Notebooks/Model_Develoment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Restaurants receive thousands of customer reviews, but star ratings alone fail to explain why ratings increase or decline. Reviews often contain rich information about food quality, service, pricing, ambience, and operational factors, yet this information remains unstructured and difficult to analyze at scale. This limits restaurants’ ability to identify the drivers of customer satisfaction and dissatisfaction

Notebook 1: Model Development

This notebook develops models for sentiment prediction and aspect extraction

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
print("PySpark installed and imported successfully!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("YelpAnalysis") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()


Loading the data

In [ ]:
business_path = "/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json"

business_df = spark.read.json(business_path)
business_df.printSchema()


In [ ]:
review_path = "/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json"

reviews_df = spark.read.json(review_path)
reviews_df.printSchema()


Cleand and Filter Data

In [ ]:
#mandatory
from pyspark.sql.functions import col

business_df_clean = business_df.filter(
    col("business_id").isNotNull() &
    col("categories").isNotNull()
)


In [ ]:
#Only need restaurant data
restaurants_df = business_df_clean.filter(
    col("categories").contains("Restaurants")
)

In [ ]:
#mandatory
reviews_df_clean = reviews_df.filter(
    col("review_id").isNotNull() &
    col("business_id").isNotNull() &
    col("text").isNotNull() &
    col("stars").isNotNull()
)


In [ ]:
#Sample Review
reviews_sample = reviews_df_clean.sample(fraction=0.05, seed=42)

In [ ]:
#Only restaurent reviews are required
restaurant_reviews_df = reviews_sample.join(
    restaurants_df.select("business_id"),
    on="business_id",
    how="inner"
)



Select Useful Fields for Sentiment Predeiction

In [ ]:
sentiment_df = restaurant_reviews_df.select("review_id", "business_id", "text", "stars")

In [ ]:
from pyspark.sql.functions import col, when

sentiment_df = sentiment_df.withColumn(
    "label",
    when(col("stars") <= 2, 0)  # Negative
    .when(col("stars") == 3, 1)  # Neutral
    .otherwise(2)  # Positive
)



In [ ]:
!pip install transformers datasets accelerate


Convert to Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Keep only relevant columns
hf_dataset = Dataset.from_spark(sentiment_df.select("text", "label"))

Tokenize

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

hf_dataset = hf_dataset.map(tokenize, batched=True)
hf_dataset = hf_dataset.remove_columns(["text"])
hf_dataset.set_format("torch")



Train / Evaluation Split

In [ ]:
hf_dataset = hf_dataset.shuffle(seed=42)

split_index = int(0.8 * len(hf_dataset))
train_dataset = hf_dataset.select(range(split_index))
eval_dataset = hf_dataset.select(range(split_index, len(hf_dataset)))


Initialize DistilBERT for classification

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=3  # Negative, Neutral, Positive
)



Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)


Evaluate arguments

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

Initialize Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)


Train Model

In [ ]:
trainer.train()


Evaluate

In [ ]:
results = trainer.evaluate()
print(results)


save sentiment prediction model

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/restaurant_sentiment_model"

trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)



ASPECT EXTRACTION

Get reviews + business_id

In [ ]:
reviews_pdf = sentiment_df.select("business_id", "text").toPandas()[:10000]
reviews_texts = reviews_pdf["text"].tolist()

In [ ]:
pip install -U bertopic

aspect extraction

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

topic_model = BERTopic(
    embedding_model=embedding_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(reviews_texts)

#SAVE MODEL

MODEL_PATH = "/content/drive/MyDrive/restaurant_sentiment_model/aspect"
topic_model.save(
    MODEL_PATH,
    serialization="pickle"
)

